In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
df=pd.read_csv("Real_Estate.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [15]:
#clean column names
df.columns=(
    df.columns.str.strip()
              .str.replace(' ','_')
              .str.replace('[^A-Za-z0-9_]+', '', regex=True)
)
print("Cleaned columns",df.columns)

Cleaned columns Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony', 'price'],
      dtype='object')


In [16]:
#handle missing values
df.isnull().sum()

,0
area_type,0
availability,0
location,1
size,16
society,5502
total_sqft,0
bath,73
balcony,609
price,0


In [18]:
#df['location'] = df['location'].fillna(df['location'].mode()[0])
df['size'] = df['size'].fillna(df['size'].mode()[0])
df['society'] = df['society'].fillna('Unknown')
df['bath'] = df['bath'].fillna(df['bath'].median())
df['balcony'] = df['balcony'].fillna(df['balcony'].median())


In [19]:
#Filter & Subset the Data
if 'Property_Type' in df.columns and 'Year_Sold' in df.columns:
    df_filtered = df[(df['Property_Type'] == 'Residential') & (df['Year_Sold'] >= 2020)]
else:
    df_filtered = df.copy()

print(f" Filtered Data Shape: {df_filtered.shape}")


 Filtered Data Shape: (13320, 9)


In [20]:
#Handle Categorical Variables
from sklearn.preprocessing import LabelEncoder

categorical_cols = df_filtered.select_dtypes(include=['object']).columns
le = LabelEncoder()

for col in categorical_cols:
    df_filtered[col] = le.fit_transform(df_filtered[col])
print(categorical_cols.tolist())

['area_type', 'availability', 'location', 'size', 'society', 'total_sqft']


In [21]:
#Aggregate Data
avg_price_by_area = df_filtered.groupby('location')['price'].mean().reset_index()
avg_price_by_area.rename(columns={'price': 'Average_Sale_Price'}, inplace=True)
print(" Average Sale Price by Location:")
print(avg_price_by_area.head())

 Average Sale Price by Location:
   location  Average_Sale_Price
0         0           16.000000
1         1           35.000000
2         2           50.000000
3         3           22.890000
4         4          124.833333


In [22]:
#Outlier Detection & Handling (IQR Method - Direct)
Q1 = df_filtered['price'].quantile(0.25)
Q3 = df_filtered['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

df_no_outliers = df_filtered[
    (df_filtered['price'] >= lower_bound) &
    (df_filtered['price'] <= upper_bound)
]

In [23]:
print("Data Shape Before Removing Outliers:", df_filtered.shape)
print("Data Shape After Removing Outliers:", df_no_outliers.shape)


Data Shape Before Removing Outliers: (13320, 9)
Data Shape After Removing Outliers: (12044, 9)


In [24]:
#final clean dataset
print("Final Cleaned Data Ready for Analysis:")
print(df_no_outliers.head())


Final Cleaned Data Ready for Analysis:
   area_type  availability  location  size  society  total_sqft  bath  \
0          3            40       419    13      464          70   2.0   
1          2            80       317    19     2439        1288   5.0   
2          0            80      1179    16     2492         514   2.0   
3          3            80       757    16     2186         602   3.0   
4          3            80       716    13     2492         239   2.0   

   balcony   price  
0      1.0   39.07  
1      3.0  120.00  
2      3.0   62.00  
3      1.0   95.00  
4      1.0   51.00  


In [25]:
#Save the cleaned dataset
df_no_outliers.to_csv("Cleaned_RealEstate.csv", index=False)